# Scoring Function

To improve efficiency: pre-allocate df, use only big score, and calculate the other df after the fact
<br>
To improve flexibility: Extract columns titles and use those titles for DF columns

In [ ]:
import numpy as np
import pandas as pd
from pandas import DataFrame as df

### Helper Function

In [ ]:
# Get DataFrame from csv files
def df_data(algo, epoch, batch_size, lr):
    fname = algo + "-gridhunter-0_epochs" + str(epoch) + "_batch_size" + str(batch_size) + "_lr" + str(lr) + "_.csv"
    file_name = "C:/Users/drago/OneDrive - UW/Ratliff Research/gridhunter_data/"+ fname
    simple_data = pd.read_csv(file_name)
    return simple_data

### Main Scoring Function

In [1]:
# Calculates Score in each epoch, returns dataframe of results total, batch, and epoch scores

def score(algos, epochs, batch_sizes, lrs):
    big_score_df = df({'Algo': [], 'Epochs': [], 'Batch Size': [], 'Learning Rate': [], 'Epoch': [], 'Return Difference': []})
    score_df = df({'Algo': [], 'Epochs': [], 'Batch Size': [], 'Learning Rate': [], 'ag0_score': [], 'ag1_score': []})
    batch_df = df({'Algo': [], 'Epochs': [], 'Batch Size': [], 'batch_ag0_score': [], 'batch_ag1_score': []})
    epoch_df = df({'Algo': [], 'Epochs': [], 'epoch_ag0_score': [], 'epoch_ag1_score': []})
    for algo in algos:
        ag0_score = 0
        ag1_score = 0
        count = 0
        for epoch in epochs:
            epoch_count = count
            epoch_ag0_score = ag0_score
            epoch_ag1_score = ag1_score
            for batch in batch_sizes:
                batch_count = count
                batch_ag0_score = ag0_score
                batch_ag1_score = ag1_score
                for lr in lrs:
                    data_df = df_data(algo, epoch, batch, lr)
                    data = data_df[['agent0_avg_rets', 'agent1_avg_rets']]
                    return_data = df.to_numpy(data)
                    trials = len(return_data[:,0])
                    lr_ag0_score = ag0_score
                    lr_ag1_score = ag1_score
                    for idx in np.arange(0,trials):
                        return_diff = 0
                        count += 1
                        if (return_data[idx,0] > return_data[idx,1]):
                            ag0_score += 1
                        else:
                            ag1_score += 1
                        return_diff = return_data[idx,0] - return_data[idx,1]
                        big_score_df.loc[len(big_score_df.index)] = [algo, epoch, batch, lr, idx, return_diff] 
                    lr_ag0_score = (ag0_score - lr_ag0_score) / trials
                    lr_ag1_score = (ag1_score - lr_ag1_score) / trials
                    score_df.loc[len(score_df.index)] = [algo, epoch, batch, lr, lr_ag0_score, lr_ag1_score] 
                batch_ag0_score = (ag0_score - batch_ag0_score) / (count - batch_count)
                batch_ag1_score = (ag1_score - batch_ag1_score) / (count - batch_count)
                batch_df.loc[len(batch_df.index)] = [algo, epoch, batch, batch_ag0_score, batch_ag1_score] 
                # print('Batch Score: algo = '+ algo + ' Batch = ' + str(batch) + ' ag0_score = ' + str(round(batch_ag0_score,5)) +   ' ag1_score = ' + str(round(batch_ag1_score,5)))
            epoch_ag0_score = (ag0_score - epoch_ag0_score) / (count - epoch_count)
            epoch_ag1_score = (ag1_score - epoch_ag1_score) / (count - epoch_count)
            epoch_df.loc[len(epoch_df.index)] = [algo, epoch, epoch_ag0_score, epoch_ag1_score] 
            # print('Epoch Score: algo = '+ algo + ' Epoch = ' + str(epoch) + ' ag0_score = ' + str(round(epoch_ag0_score,5)) +   ' ag1_score = ' + str(round(epoch_ag1_score,5)))
        ag0_score /= count
        ag1_score /= count
        print('    Total Score: algo = '+ algo + ' ag0_score = ' + str(round(ag0_score,5)) +   ' ag1_score = ' + str(round(ag1_score,5)))
        
    return big_score_df, score_df, batch_df, epoch_df

In [ ]:
# Example Use
algos = ['SimplePG', 'StackPG']
epochs = [50, 100, 500]
batch_sizes = [100, 500, 1000, 5000]
lrs = [0.001, 0.01, 0.1]
big_score_df, score_df, batch_df, epoch_df = score(algos, epochs, batch_sizes, lrs)

big_score_df.to_pickle("C:/Users/drago/OneDrive - UW/Ratliff Research/gridhunter_data/big_score_df.pkl")